# Day 06: High-Frequency Trading (HFT) Strategies

## Week 21: Market Microstructure

---

### Learning Objectives
- Understand the fundamentals of high-frequency trading and market microstructure
- Build and simulate limit order book dynamics
- Implement market making strategies with inventory management
- Develop statistical arbitrage strategies at tick frequency
- Create optimal execution algorithms (TWAP/VWAP)
- Detect market manipulation patterns (momentum ignition, spoofing)
- Backtest HFT strategies with realistic transaction costs

### Prerequisites
- Understanding of order book mechanics
- Familiarity with time series analysis
- Python proficiency with numpy, pandas

---

## 1. Import Required Libraries

Import necessary libraries for high-frequency trading analysis, simulation, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize
from collections import defaultdict, deque
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
from enum import Enum
import time
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Market Data Simulation and Tick Data Processing

### 2.1 Understanding Tick Data

High-frequency trading operates on tick-by-tick data with microsecond precision. Key characteristics:
- **Timestamps**: Microsecond or nanosecond precision
- **Bid/Ask Prices**: Best bid and offer at each moment
- **Trade Prices**: Actual execution prices
- **Volumes**: Order sizes at each price level

Let's generate realistic synthetic tick data:

In [ ]:
class TickDataGenerator:
    """Generate realistic tick-by-tick market data."""
    
    def __init__(self, 
                 initial_price: float = 100.0,
                 tick_size: float = 0.01,
                 avg_spread_ticks: int = 2,
                 volatility: float = 0.0002,  # Per-tick volatility
                 mean_reversion: float = 0.1):
        self.initial_price = initial_price
        self.tick_size = tick_size
        self.avg_spread_ticks = avg_spread_ticks
        self.volatility = volatility
        self.mean_reversion = mean_reversion
        
    def generate_ticks(self, n_ticks: int = 10000) -> pd.DataFrame:
        """Generate n_ticks of market data."""
        
        # Generate timestamps (microsecond precision, ~100μs between ticks on average)
        base_time = pd.Timestamp('2024-01-15 09:30:00')
        inter_arrival_times = np.random.exponential(100, n_ticks)  # microseconds
        timestamps = base_time + pd.to_timedelta(np.cumsum(inter_arrival_times), unit='us')
        
        # Generate mid-price using Ornstein-Uhlenbeck process (mean-reverting)
        mid_prices = np.zeros(n_ticks)
        mid_prices[0] = self.initial_price
        
        for i in range(1, n_ticks):
            # Mean-reverting random walk
            drift = self.mean_reversion * (self.initial_price - mid_prices[i-1])
            shock = self.volatility * np.random.randn()
            mid_prices[i] = mid_prices[i-1] + drift + shock
        
        # Round to tick size
        mid_prices = np.round(mid_prices / self.tick_size) * self.tick_size
        
        # Generate dynamic spreads (wider during volatility)
        returns = np.diff(mid_prices, prepend=mid_prices[0])
        rolling_vol = pd.Series(returns).rolling(100, min_periods=1).std().values
        normalized_vol = rolling_vol / (rolling_vol.mean() + 1e-10)
        spread_ticks = np.maximum(1, np.round(self.avg_spread_ticks * normalized_vol)).astype(int)
        
        # Calculate bid/ask prices
        half_spread = (spread_ticks * self.tick_size) / 2
        bid_prices = mid_prices - half_spread
        ask_prices = mid_prices + half_spread
        
        # Generate volumes (power-law distribution for realistic heavy tails)
        bid_sizes = np.round(100 * np.random.pareto(1.5, n_ticks) + 100).astype(int)
        ask_sizes = np.round(100 * np.random.pareto(1.5, n_ticks) + 100).astype(int)
        
        # Generate trade direction (-1 = sell, +1 = buy, 0 = no trade)
        trade_prob = 0.3  # 30% of ticks have trades
        trade_direction = np.where(
            np.random.random(n_ticks) < trade_prob,
            np.random.choice([-1, 1], n_ticks),
            0
        )
        
        # Trade prices (at bid for sells, at ask for buys)
        trade_prices = np.where(
            trade_direction == 1, ask_prices,
            np.where(trade_direction == -1, bid_prices, np.nan)
        )
        
        # Trade sizes (subset of available size)
        trade_sizes = np.where(
            trade_direction == 1, 
            np.minimum(np.round(ask_sizes * np.random.uniform(0.1, 0.5, n_ticks)).astype(int), ask_sizes),
            np.where(
                trade_direction == -1,
                np.minimum(np.round(bid_sizes * np.random.uniform(0.1, 0.5, n_ticks)).astype(int), bid_sizes),
                0
            )
        )
        
        return pd.DataFrame({
            'timestamp': timestamps,
            'bid_price': bid_prices,
            'ask_price': ask_prices,
            'bid_size': bid_sizes,
            'ask_size': ask_sizes,
            'mid_price': mid_prices,
            'spread': ask_prices - bid_prices,
            'trade_direction': trade_direction,
            'trade_price': trade_prices,
            'trade_size': trade_sizes
        })

# Generate tick data
generator = TickDataGenerator(
    initial_price=100.0,
    tick_size=0.01,
    avg_spread_ticks=2,
    volatility=0.0003
)

tick_data = generator.generate_ticks(50000)
print(f"Generated {len(tick_data)} ticks")
print(f"\nTime span: {tick_data['timestamp'].iloc[-1] - tick_data['timestamp'].iloc[0]}")
print(f"\nTick Data Summary:")
tick_data.describe()

In [ ]:
# Visualize tick data
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Plot 1: Price evolution with bid-ask
ax1 = axes[0, 0]
sample = tick_data.iloc[:1000]
ax1.fill_between(range(len(sample)), sample['bid_price'], sample['ask_price'], 
                  alpha=0.3, color='blue', label='Bid-Ask Spread')
ax1.plot(sample['mid_price'], color='black', linewidth=0.5, label='Mid Price')
ax1.set_xlabel('Tick')
ax1.set_ylabel('Price')
ax1.set_title('Price Evolution with Bid-Ask Spread (First 1000 ticks)')
ax1.legend()

# Plot 2: Spread distribution
ax2 = axes[0, 1]
spread_cents = tick_data['spread'] * 100
ax2.hist(spread_cents, bins=50, edgecolor='black', alpha=0.7)
ax2.axvline(spread_cents.mean(), color='red', linestyle='--', label=f'Mean: {spread_cents.mean():.2f}¢')
ax2.set_xlabel('Spread (cents)')
ax2.set_ylabel('Frequency')
ax2.set_title('Bid-Ask Spread Distribution')
ax2.legend()

# Plot 3: Trade direction over time
ax3 = axes[1, 0]
trades_only = tick_data[tick_data['trade_direction'] != 0].iloc[:500]
colors = ['red' if d < 0 else 'green' for d in trades_only['trade_direction']]
ax3.scatter(range(len(trades_only)), trades_only['trade_price'], c=colors, s=trades_only['trade_size']/50, alpha=0.6)
ax3.set_xlabel('Trade Index')
ax3.set_ylabel('Trade Price')
ax3.set_title('Trade Prices (Green=Buy, Red=Sell, Size=Volume)')

# Plot 4: Volume distribution
ax4 = axes[1, 1]
ax4.hist(tick_data['bid_size'], bins=50, alpha=0.5, label='Bid Size', edgecolor='black')
ax4.hist(tick_data['ask_size'], bins=50, alpha=0.5, label='Ask Size', edgecolor='black')
ax4.set_xlabel('Size')
ax4.set_ylabel('Frequency')
ax4.set_title('Order Size Distribution')
ax4.legend()
ax4.set_xlim(0, 2000)

# Plot 5: Inter-arrival times
ax5 = axes[2, 0]
inter_arrival = tick_data['timestamp'].diff().dt.total_seconds() * 1e6  # microseconds
ax5.hist(inter_arrival.dropna(), bins=50, edgecolor='black', alpha=0.7)
ax5.axvline(inter_arrival.mean(), color='red', linestyle='--', label=f'Mean: {inter_arrival.mean():.1f}μs')
ax5.set_xlabel('Inter-arrival Time (μs)')
ax5.set_ylabel('Frequency')
ax5.set_title('Tick Inter-arrival Time Distribution')
ax5.legend()

# Plot 6: Returns distribution
ax6 = axes[2, 1]
returns = tick_data['mid_price'].pct_change().dropna() * 10000  # basis points
ax6.hist(returns, bins=100, edgecolor='black', alpha=0.7)
ax6.axvline(0, color='red', linestyle='--')
ax6.set_xlabel('Returns (bps)')
ax6.set_ylabel('Frequency')
ax6.set_title('Tick-by-Tick Returns Distribution')
print(f"Kurtosis: {stats.kurtosis(returns):.2f} (Normal = 0)")

plt.tight_layout()
plt.show()

## 3. Order Book Modeling and Analysis

### 3.1 Limit Order Book (LOB) Simulator

The Limit Order Book is the central data structure in HFT. It maintains:
- **Bid side**: Buy orders sorted by price (highest first)
- **Ask side**: Sell orders sorted by price (lowest first)
- **Queue priority**: Orders at same price level sorted by time (FIFO)

In [ ]:
class OrderSide(Enum):
    BID = 'bid'
    ASK = 'ask'

@dataclass
class Order:
    """Represents a single order in the book."""
    order_id: int
    price: float
    size: int
    side: OrderSide
    timestamp: float
    
@dataclass 
class PriceLevel:
    """Represents a single price level with queue of orders."""
    price: float
    orders: List[Order] = field(default_factory=list)
    
    @property
    def total_size(self) -> int:
        return sum(o.size for o in self.orders)
    
    def add_order(self, order: Order):
        self.orders.append(order)
        
    def remove_order(self, order_id: int) -> Optional[Order]:
        for i, order in enumerate(self.orders):
            if order.order_id == order_id:
                return self.orders.pop(i)
        return None

class LimitOrderBook:
    """
    A simplified Limit Order Book implementation.
    Maintains bid and ask sides with price-time priority.
    """
    
    def __init__(self, tick_size: float = 0.01):
        self.tick_size = tick_size
        self.bids: Dict[float, PriceLevel] = {}  # price -> PriceLevel
        self.asks: Dict[float, PriceLevel] = {}
        self.order_map: Dict[int, Tuple[OrderSide, float]] = {}  # order_id -> (side, price)
        self.next_order_id = 0
        self.trades: List[Dict] = []
        
    def _round_price(self, price: float) -> float:
        return round(price / self.tick_size) * self.tick_size
    
    @property
    def best_bid(self) -> Optional[float]:
        return max(self.bids.keys()) if self.bids else None
    
    @property
    def best_ask(self) -> Optional[float]:
        return min(self.asks.keys()) if self.asks else None
    
    @property
    def mid_price(self) -> Optional[float]:
        if self.best_bid and self.best_ask:
            return (self.best_bid + self.best_ask) / 2
        return None
    
    @property
    def spread(self) -> Optional[float]:
        if self.best_bid and self.best_ask:
            return self.best_ask - self.best_bid
        return None
    
    @property
    def microprice(self) -> Optional[float]:
        """Volume-weighted mid-price - better fair value estimate."""
        if not self.best_bid or not self.best_ask:
            return None
        bid_size = self.bids[self.best_bid].total_size
        ask_size = self.asks[self.best_ask].total_size
        total_size = bid_size + ask_size
        if total_size == 0:
            return self.mid_price
        # Weight towards the side with less volume (more pressure from opposite side)
        return (self.best_bid * ask_size + self.best_ask * bid_size) / total_size
    
    def add_order(self, side: OrderSide, price: float, size: int, timestamp: float = None) -> int:
        """Add a new order to the book. Returns order_id."""
        if timestamp is None:
            timestamp = time.time()
            
        price = self._round_price(price)
        order_id = self.next_order_id
        self.next_order_id += 1
        
        order = Order(order_id, price, size, side, timestamp)
        
        # Check for crossing (marketable order)
        remaining_size = self._match_order(order)
        
        if remaining_size > 0:
            # Add remaining to book
            order.size = remaining_size
            book = self.bids if side == OrderSide.BID else self.asks
            
            if price not in book:
                book[price] = PriceLevel(price)
            book[price].add_order(order)
            self.order_map[order_id] = (side, price)
            
        return order_id
    
    def _match_order(self, order: Order) -> int:
        """Match incoming order against resting orders. Returns remaining size."""
        if order.side == OrderSide.BID:
            # Buy order matches against asks
            opposite_book = self.asks
            price_check = lambda ask_price: order.price >= ask_price
            get_best = lambda: min(opposite_book.keys()) if opposite_book else None
        else:
            # Sell order matches against bids
            opposite_book = self.bids
            price_check = lambda bid_price: order.price <= bid_price
            get_best = lambda: max(opposite_book.keys()) if opposite_book else None
            
        remaining = order.size
        
        while remaining > 0:
            best_price = get_best()
            if best_price is None or not price_check(best_price):
                break
                
            level = opposite_book[best_price]
            
            while remaining > 0 and level.orders:
                resting_order = level.orders[0]
                fill_size = min(remaining, resting_order.size)
                
                # Record trade
                self.trades.append({
                    'price': best_price,
                    'size': fill_size,
                    'aggressor_side': order.side.value,
                    'timestamp': order.timestamp
                })
                
                remaining -= fill_size
                resting_order.size -= fill_size
                
                if resting_order.size == 0:
                    level.orders.pop(0)
                    if resting_order.order_id in self.order_map:
                        del self.order_map[resting_order.order_id]
                        
            # Remove empty price level
            if not level.orders:
                del opposite_book[best_price]
                
        return remaining
    
    def cancel_order(self, order_id: int) -> bool:
        """Cancel an order by ID."""
        if order_id not in self.order_map:
            return False
            
        side, price = self.order_map[order_id]
        book = self.bids if side == OrderSide.BID else self.asks
        
        if price in book:
            book[price].remove_order(order_id)
            if not book[price].orders:
                del book[price]
                
        del self.order_map[order_id]
        return True
    
    def get_depth(self, levels: int = 5) -> Dict:
        """Get order book depth up to n levels."""
        bid_prices = sorted(self.bids.keys(), reverse=True)[:levels]
        ask_prices = sorted(self.asks.keys())[:levels]
        
        return {
            'bids': [(p, self.bids[p].total_size) for p in bid_prices],
            'asks': [(p, self.asks[p].total_size) for p in ask_prices]
        }
    
    def get_order_book_imbalance(self, levels: int = 5) -> float:
        """
        Calculate order book imbalance.
        Returns value between -1 (all asks) and +1 (all bids).
        Positive imbalance suggests upward price pressure.
        """
        depth = self.get_depth(levels)
        bid_volume = sum(size for _, size in depth['bids'])
        ask_volume = sum(size for _, size in depth['asks'])
        total = bid_volume + ask_volume
        
        if total == 0:
            return 0.0
        return (bid_volume - ask_volume) / total

# Initialize and populate order book
lob = LimitOrderBook(tick_size=0.01)

# Seed the order book with realistic depth
np.random.seed(42)
base_price = 100.0

# Add bid orders (prices below 100)
for i in range(10):
    price = base_price - 0.01 * (i + 1)
    n_orders = np.random.randint(3, 8)
    for _ in range(n_orders):
        size = int(np.random.pareto(1.5) * 100 + 100)
        lob.add_order(OrderSide.BID, price, size)

# Add ask orders (prices above 100)
for i in range(10):
    price = base_price + 0.01 * (i + 1)
    n_orders = np.random.randint(3, 8)
    for _ in range(n_orders):
        size = int(np.random.pareto(1.5) * 100 + 100)
        lob.add_order(OrderSide.ASK, price, size)

print("Order Book Initialized")
print(f"Best Bid: ${lob.best_bid:.2f}")
print(f"Best Ask: ${lob.best_ask:.2f}")
print(f"Spread: ${lob.spread:.4f} ({lob.spread/lob.mid_price*10000:.2f} bps)")
print(f"Mid Price: ${lob.mid_price:.4f}")
print(f"Microprice: ${lob.microprice:.4f}")
print(f"Order Book Imbalance: {lob.get_order_book_imbalance():.3f}")

In [ ]:
# Visualize order book depth
def plot_order_book(lob: LimitOrderBook, levels: int = 10):
    """Visualize order book depth."""
    depth = lob.get_depth(levels)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Bid side (buy orders)
    bid_prices = [p for p, _ in depth['bids']]
    bid_sizes = [s for _, s in depth['bids']]
    bid_cumsum = np.cumsum(bid_sizes)
    
    # Ask side (sell orders)
    ask_prices = [p for p, _ in depth['asks']]
    ask_sizes = [s for _, s in depth['asks']]
    ask_cumsum = np.cumsum(ask_sizes)
    
    # Plot cumulative depth
    ax.fill_between(bid_prices, 0, bid_cumsum, alpha=0.5, color='green', 
                    step='pre', label='Bid Depth')
    ax.fill_between(ask_prices, 0, ask_cumsum, alpha=0.5, color='red', 
                    step='pre', label='Ask Depth')
    
    # Plot individual levels
    ax.bar(bid_prices, bid_sizes, width=0.008, color='green', alpha=0.8, edgecolor='darkgreen')
    ax.bar(ask_prices, ask_sizes, width=0.008, color='red', alpha=0.8, edgecolor='darkred')
    
    # Mark mid price and microprice
    ax.axvline(lob.mid_price, color='black', linestyle='--', linewidth=2, label=f'Mid: ${lob.mid_price:.2f}')
    ax.axvline(lob.microprice, color='blue', linestyle=':', linewidth=2, label=f'Micro: ${lob.microprice:.2f}')
    
    ax.set_xlabel('Price ($)')
    ax.set_ylabel('Size (shares)')
    ax.set_title(f'Limit Order Book Depth\nSpread: ${lob.spread:.2f} | Imbalance: {lob.get_order_book_imbalance():.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_order_book(lob)

## 4. Latency Measurement and Optimization

### 4.1 Understanding Latency in HFT

In HFT, latency is measured in microseconds or nanoseconds. Key latency components:
- **Network latency**: Time for data to travel between systems
- **Processing latency**: Time to process market data and make decisions
- **Execution latency**: Time from order generation to exchange acknowledgment

Even a few microseconds can determine profit vs. loss in competitive HFT.

In [ ]:
class LatencyProfiler:
    """Profile and analyze latency in trading operations."""
    
    def __init__(self):
        self.measurements: Dict[str, List[float]] = defaultdict(list)
        
    def measure(self, operation_name: str):
        """Context manager for measuring operation latency."""
        class Timer:
            def __init__(self, profiler, name):
                self.profiler = profiler
                self.name = name
                self.start = None
                
            def __enter__(self):
                self.start = time.perf_counter_ns()
                return self
                
            def __exit__(self, *args):
                elapsed_ns = time.perf_counter_ns() - self.start
                self.profiler.measurements[self.name].append(elapsed_ns / 1000)  # Convert to μs
                
        return Timer(self, operation_name)
    
    def get_stats(self, operation_name: str) -> Dict:
        """Get latency statistics for an operation."""
        if operation_name not in self.measurements:
            return {}
        
        data = np.array(self.measurements[operation_name])
        return {
            'count': len(data),
            'mean_us': np.mean(data),
            'std_us': np.std(data),
            'min_us': np.min(data),
            'max_us': np.max(data),
            'p50_us': np.percentile(data, 50),
            'p95_us': np.percentile(data, 95),
            'p99_us': np.percentile(data, 99)
        }
    
    def summary(self) -> pd.DataFrame:
        """Get summary of all measured operations."""
        rows = []
        for name in self.measurements:
            stats = self.get_stats(name)
            stats['operation'] = name
            rows.append(stats)
        return pd.DataFrame(rows).set_index('operation')

# Demonstrate latency profiling
profiler = LatencyProfiler()

# Simulate various trading operations
def simulate_market_data_processing(n: int = 1000):
    """Simulate processing incoming market data."""
    data = np.random.randn(n)
    mean = np.mean(data)
    std = np.std(data)
    return mean, std

def simulate_signal_calculation(prices: np.ndarray):
    """Simulate calculating a trading signal."""
    # Simple momentum signal
    returns = np.diff(prices) / prices[:-1]
    ema_fast = pd.Series(returns).ewm(span=10).mean().iloc[-1]
    ema_slow = pd.Series(returns).ewm(span=50).mean().iloc[-1]
    return ema_fast - ema_slow

def simulate_order_generation(signal: float, price: float):
    """Simulate generating an order based on signal."""
    if signal > 0.001:
        return {'side': 'buy', 'price': price * 0.999, 'size': 100}
    elif signal < -0.001:
        return {'side': 'sell', 'price': price * 1.001, 'size': 100}
    return None

# Run profiling
prices = tick_data['mid_price'].values[:1000]

for i in range(1000):
    with profiler.measure('market_data_processing'):
        simulate_market_data_processing(100)
    
    with profiler.measure('signal_calculation'):
        signal = simulate_signal_calculation(prices)
    
    with profiler.measure('order_generation'):
        order = simulate_order_generation(signal, prices[-1])
    
    with profiler.measure('order_book_update'):
        lob.add_order(OrderSide.BID, 99.95, 100)
        lob.cancel_order(lob.next_order_id - 1)

# Display latency statistics
print("Latency Statistics (microseconds)")
print("=" * 70)
latency_summary = profiler.summary()
print(latency_summary[['count', 'mean_us', 'p50_us', 'p95_us', 'p99_us', 'max_us']].round(2))

In [ ]:
# Visualize latency distribution
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

operations = list(profiler.measurements.keys())
colors = ['blue', 'green', 'orange', 'red']

for ax, op, color in zip(axes.flat, operations, colors):
    data = profiler.measurements[op]
    ax.hist(data, bins=50, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(np.percentile(data, 50), color='black', linestyle='--', label=f'p50: {np.percentile(data, 50):.2f}μs')
    ax.axvline(np.percentile(data, 99), color='red', linestyle='--', label=f'p99: {np.percentile(data, 99):.2f}μs')
    ax.set_xlabel('Latency (μs)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{op.replace("_", " ").title()}')
    ax.legend()

plt.suptitle('Latency Distribution by Operation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Key Insight: In production HFT systems, these operations would be")
print("   optimized to run in nanoseconds using C++/FPGA hardware.")

## 5. Market Making Strategy Implementation

### 5.1 Understanding Market Making

Market makers provide liquidity by continuously quoting bid and ask prices. They profit from:
1. **Spread capture**: Buying at bid, selling at ask
2. **Rebates**: Exchanges pay for providing liquidity

Key risks:
- **Adverse selection**: Trading against informed traders
- **Inventory risk**: Accumulating directional positions
- **Toxic flow**: High-frequency traders picking off stale quotes

In [ ]:
class MarketMaker:
    """
    Avellaneda-Stoikov Market Making Model.
    
    Optimal bid/ask quotes considering:
    - Inventory risk
    - Volatility
    - Time horizon
    """
    
    def __init__(self,
                 gamma: float = 0.1,      # Risk aversion parameter
                 sigma: float = 0.02,     # Volatility (per period)
                 k: float = 1.5,          # Order arrival intensity
                 A: float = 140,          # Order arrival constant
                 max_inventory: int = 100,# Maximum inventory position
                 tick_size: float = 0.01):
        
        self.gamma = gamma
        self.sigma = sigma
        self.k = k
        self.A = A
        self.max_inventory = max_inventory
        self.tick_size = tick_size
        
        # State
        self.inventory = 0
        self.cash = 0
        self.trades = []
        self.quotes_history = []
        self.pnl_history = []
        
    def calculate_reservation_price(self, mid_price: float, T: float = 1.0, t: float = 0.0) -> float:
        """
        Calculate reservation price (fair value considering inventory).
        
        r(s, q, t) = s - q * gamma * sigma^2 * (T - t)
        
        Where:
        - s: mid price
        - q: inventory
        - gamma: risk aversion
        - sigma: volatility
        - T-t: time remaining
        """
        time_remaining = T - t
        return mid_price - self.inventory * self.gamma * (self.sigma ** 2) * time_remaining
    
    def calculate_optimal_spread(self, T: float = 1.0, t: float = 0.0) -> float:
        """
        Calculate optimal spread width.
        
        delta = gamma * sigma^2 * (T - t) + (2/gamma) * ln(1 + gamma/k)
        """
        time_remaining = T - t
        spread = (self.gamma * (self.sigma ** 2) * time_remaining + 
                  (2 / self.gamma) * np.log(1 + self.gamma / self.k))
        return max(spread, 2 * self.tick_size)  # Minimum 2-tick spread
    
    def calculate_inventory_skew(self) -> float:
        """
        Calculate quote skewing based on inventory.
        Skew quotes to reduce inventory risk.
        """
        # Normalize inventory to [-1, 1]
        normalized_inv = self.inventory / self.max_inventory
        # Skew increases with inventory magnitude
        return normalized_inv * self.sigma * 0.5
    
    def generate_quotes(self, mid_price: float, T: float = 1.0, t: float = 0.0) -> Tuple[float, float]:
        """Generate optimal bid and ask quotes."""
        reservation_price = self.calculate_reservation_price(mid_price, T, t)
        half_spread = self.calculate_optimal_spread(T, t) / 2
        skew = self.calculate_inventory_skew()
        
        # Apply inventory skew
        bid_price = reservation_price - half_spread - skew
        ask_price = reservation_price + half_spread - skew
        
        # Round to tick size
        bid_price = np.floor(bid_price / self.tick_size) * self.tick_size
        ask_price = np.ceil(ask_price / self.tick_size) * self.tick_size
        
        # Ensure minimum spread
        if ask_price <= bid_price:
            ask_price = bid_price + self.tick_size
            
        self.quotes_history.append({
            'mid_price': mid_price,
            'reservation_price': reservation_price,
            'bid': bid_price,
            'ask': ask_price,
            'spread': ask_price - bid_price,
            'inventory': self.inventory,
            't': t
        })
        
        return bid_price, ask_price
    
    def execute_trade(self, side: str, price: float, size: int, timestamp: float):
        """Record a trade execution."""
        if side == 'buy':  # MM bought (someone sold to us)
            self.inventory += size
            self.cash -= price * size
        else:  # MM sold (someone bought from us)
            self.inventory -= size
            self.cash += price * size
            
        self.trades.append({
            'side': side,
            'price': price,
            'size': size,
            'timestamp': timestamp,
            'inventory_after': self.inventory,
            'cash_after': self.cash
        })
    
    def calculate_pnl(self, current_mid_price: float) -> float:
        """Calculate mark-to-market PnL."""
        return self.cash + self.inventory * current_mid_price
    
    def reset(self):
        """Reset state for new simulation."""
        self.inventory = 0
        self.cash = 0
        self.trades = []
        self.quotes_history = []
        self.pnl_history = []

# Demonstrate market making quotes
mm = MarketMaker(gamma=0.1, sigma=0.02, k=1.5)

print("Market Making Quote Generation")
print("=" * 60)

mid_prices = [100.0, 100.05, 99.95, 100.10, 99.90]
inventories = [0, 10, -10, 50, -50]

for mid, inv in zip(mid_prices, inventories):
    mm.inventory = inv
    bid, ask = mm.generate_quotes(mid)
    spread = ask - bid
    print(f"Mid: ${mid:.2f} | Inventory: {inv:+4d} | "
          f"Bid: ${bid:.2f} | Ask: ${ask:.2f} | Spread: ${spread:.4f}")

print("\n💡 Note: With positive inventory, quotes skew lower to encourage sells")
print("   With negative inventory, quotes skew higher to encourage buys")

In [ ]:
def simulate_market_making(tick_data: pd.DataFrame, mm: MarketMaker, 
                           quote_size: int = 100) -> pd.DataFrame:
    """
    Simulate market making strategy on tick data.
    
    Assumptions:
    - MM continuously quotes bid and ask
    - Incoming trades hit MM's quotes based on direction
    - Simple fill model: full fill if size available
    """
    mm.reset()
    
    results = []
    T = len(tick_data)
    
    for i, row in tick_data.iterrows():
        t = i / T  # Normalized time
        mid_price = row['mid_price']
        
        # Generate quotes
        bid, ask = mm.generate_quotes(mid_price, T=1.0, t=t)
        
        # Check if incoming trade hits our quotes
        if row['trade_direction'] != 0:
            trade_price = row['trade_price']
            trade_size = min(row['trade_size'], quote_size)
            
            # Buy order hits our ask (we sell)
            if row['trade_direction'] == 1 and trade_price >= ask:
                if mm.inventory > -mm.max_inventory:
                    mm.execute_trade('sell', ask, trade_size, i)
                    
            # Sell order hits our bid (we buy)
            elif row['trade_direction'] == -1 and trade_price <= bid:
                if mm.inventory < mm.max_inventory:
                    mm.execute_trade('buy', bid, trade_size, i)
        
        # Record PnL
        pnl = mm.calculate_pnl(mid_price)
        mm.pnl_history.append({
            'tick': i,
            'mid_price': mid_price,
            'pnl': pnl,
            'inventory': mm.inventory,
            'cash': mm.cash
        })
        
    return pd.DataFrame(mm.pnl_history)

# Run market making simulation
mm = MarketMaker(gamma=0.1, sigma=0.02, k=1.5, max_inventory=500)
mm_results = simulate_market_making(tick_data.head(20000), mm, quote_size=50)

# Calculate statistics
trades_df = pd.DataFrame(mm.trades)
quotes_df = pd.DataFrame(mm.quotes_history)

print("Market Making Simulation Results")
print("=" * 60)
print(f"Total Trades: {len(trades_df)}")
print(f"  - Buys: {len(trades_df[trades_df['side'] == 'buy'])}")
print(f"  - Sells: {len(trades_df[trades_df['side'] == 'sell'])}")
print(f"\nFinal State:")
print(f"  - Final Inventory: {mm.inventory}")
print(f"  - Final Cash: ${mm.cash:,.2f}")
print(f"  - Final PnL: ${mm_results['pnl'].iloc[-1]:,.2f}")
print(f"\nPnL Statistics:")
print(f"  - Max PnL: ${mm_results['pnl'].max():,.2f}")
print(f"  - Min PnL: ${mm_results['pnl'].min():,.2f}")
print(f"  - PnL Volatility: ${mm_results['pnl'].std():,.2f}")

In [ ]:
# Visualize market making performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: PnL over time
ax1 = axes[0, 0]
ax1.plot(mm_results['tick'], mm_results['pnl'], color='blue', linewidth=0.8)
ax1.axhline(0, color='black', linestyle='--', alpha=0.5)
ax1.fill_between(mm_results['tick'], 0, mm_results['pnl'], 
                  where=mm_results['pnl'] > 0, color='green', alpha=0.3)
ax1.fill_between(mm_results['tick'], 0, mm_results['pnl'], 
                  where=mm_results['pnl'] < 0, color='red', alpha=0.3)
ax1.set_xlabel('Tick')
ax1.set_ylabel('PnL ($)')
ax1.set_title('Market Making PnL Over Time')

# Plot 2: Inventory evolution
ax2 = axes[0, 1]
ax2.plot(mm_results['tick'], mm_results['inventory'], color='purple', linewidth=0.8)
ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
ax2.axhline(mm.max_inventory, color='red', linestyle=':', label='Max Inventory')
ax2.axhline(-mm.max_inventory, color='red', linestyle=':')
ax2.set_xlabel('Tick')
ax2.set_ylabel('Inventory (shares)')
ax2.set_title('Inventory Evolution')
ax2.legend()

# Plot 3: Spread distribution
ax3 = axes[1, 0]
spreads = quotes_df['spread'] * 100  # Convert to cents
ax3.hist(spreads, bins=50, color='orange', edgecolor='black', alpha=0.7)
ax3.axvline(spreads.mean(), color='red', linestyle='--', label=f'Mean: {spreads.mean():.2f}¢')
ax3.set_xlabel('Quoted Spread (cents)')
ax3.set_ylabel('Frequency')
ax3.set_title('Distribution of Quoted Spreads')
ax3.legend()

# Plot 4: PnL vs Inventory scatter
ax4 = axes[1, 1]
scatter = ax4.scatter(mm_results['inventory'], mm_results['pnl'], 
                       c=mm_results['tick'], cmap='viridis', alpha=0.3, s=5)
plt.colorbar(scatter, ax=ax4, label='Tick')
ax4.axhline(0, color='black', linestyle='--', alpha=0.5)
ax4.axvline(0, color='black', linestyle='--', alpha=0.5)
ax4.set_xlabel('Inventory')
ax4.set_ylabel('PnL ($)')
ax4.set_title('PnL vs Inventory')

plt.tight_layout()
plt.show()

## 6. Statistical Arbitrage at High Frequency

### 6.1 Pairs Trading at Tick Level

Statistical arbitrage exploits temporary mispricings between related securities. At HFT speeds:
- **Lead-lag relationships**: One asset leads another by milliseconds
- **Cross-exchange arbitrage**: Same asset priced differently across venues
- **ETF arbitrage**: ETF price vs. underlying basket

In [ ]:
def generate_correlated_assets(n_ticks: int = 10000, 
                                correlation: float = 0.95,
                                lead_lag: int = 5) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Generate two correlated assets with lead-lag relationship.
    Asset A leads Asset B by `lead_lag` ticks.
    """
    np.random.seed(42)
    
    # Generate common factor
    common_factor = np.cumsum(np.random.randn(n_ticks + lead_lag) * 0.0003)
    
    # Generate idiosyncratic noise
    noise_a = np.cumsum(np.random.randn(n_ticks) * 0.0001)
    noise_b = np.cumsum(np.random.randn(n_ticks) * 0.0001)
    
    # Asset A leads
    price_a = 100 + common_factor[:n_ticks] + noise_a
    
    # Asset B lags by `lead_lag` ticks
    price_b = 100 + common_factor[lead_lag:lead_lag + n_ticks] + noise_b
    
    # Create DataFrames
    base_time = pd.Timestamp('2024-01-15 09:30:00')
    timestamps = base_time + pd.to_timedelta(np.arange(n_ticks) * 100, unit='us')
    
    asset_a = pd.DataFrame({'timestamp': timestamps, 'price': price_a})
    asset_b = pd.DataFrame({'timestamp': timestamps, 'price': price_b})
    
    return asset_a, asset_b

class HFTPairsTrader:
    """High-frequency pairs trading strategy."""
    
    def __init__(self, 
                 lookback: int = 100,
                 entry_zscore: float = 2.0,
                 exit_zscore: float = 0.5,
                 position_size: int = 100):
        
        self.lookback = lookback
        self.entry_zscore = entry_zscore
        self.exit_zscore = exit_zscore
        self.position_size = position_size
        
        # State
        self.position = 0  # +1 = long spread, -1 = short spread
        self.trades = []
        self.spread_history = []
        
    def calculate_spread(self, price_a: float, price_b: float) -> float:
        """Calculate raw spread."""
        return price_a - price_b
    
    def calculate_zscore(self, spread: float, spread_history: List[float]) -> float:
        """Calculate z-score of current spread."""
        if len(spread_history) < self.lookback:
            return 0.0
        
        recent = spread_history[-self.lookback:]
        mean = np.mean(recent)
        std = np.std(recent)
        
        if std < 1e-8:
            return 0.0
        return (spread - mean) / std
    
    def generate_signal(self, zscore: float) -> int:
        """
        Generate trading signal based on z-score.
        
        Returns:
        +1: Enter long spread (buy A, sell B)
        -1: Enter short spread (sell A, buy B)
        0: No action
        """
        if self.position == 0:
            # Not in position - check for entry
            if zscore < -self.entry_zscore:
                return 1  # Spread too low, go long
            elif zscore > self.entry_zscore:
                return -1  # Spread too high, go short
        else:
            # In position - check for exit
            if self.position == 1 and zscore > -self.exit_zscore:
                return -1  # Exit long
            elif self.position == -1 and zscore < self.exit_zscore:
                return 1  # Exit short
                
        return 0
    
    def backtest(self, asset_a: pd.DataFrame, asset_b: pd.DataFrame) -> pd.DataFrame:
        """Run backtest on historical data."""
        results = []
        self.spread_history = []
        self.position = 0
        pnl = 0
        entry_prices = {'a': 0, 'b': 0}
        
        for i in range(len(asset_a)):
            price_a = asset_a['price'].iloc[i]
            price_b = asset_b['price'].iloc[i]
            
            spread = self.calculate_spread(price_a, price_b)
            self.spread_history.append(spread)
            
            zscore = self.calculate_zscore(spread, self.spread_history)
            signal = self.generate_signal(zscore)
            
            # Execute signal
            if signal != 0 and self.position == 0:
                # Enter position
                self.position = signal
                entry_prices = {'a': price_a, 'b': price_b}
                self.trades.append({
                    'tick': i,
                    'action': 'enter',
                    'direction': 'long' if signal == 1 else 'short',
                    'price_a': price_a,
                    'price_b': price_b,
                    'zscore': zscore
                })
            elif signal != 0 and signal != self.position:
                # Exit position
                if self.position == 1:
                    # Was long spread: bought A, sold B
                    trade_pnl = (price_a - entry_prices['a']) - (price_b - entry_prices['b'])
                else:
                    # Was short spread: sold A, bought B
                    trade_pnl = (entry_prices['a'] - price_a) - (entry_prices['b'] - price_b)
                    
                pnl += trade_pnl * self.position_size
                self.trades.append({
                    'tick': i,
                    'action': 'exit',
                    'direction': 'long' if self.position == 1 else 'short',
                    'price_a': price_a,
                    'price_b': price_b,
                    'zscore': zscore,
                    'trade_pnl': trade_pnl * self.position_size
                })
                self.position = 0
                
            results.append({
                'tick': i,
                'price_a': price_a,
                'price_b': price_b,
                'spread': spread,
                'zscore': zscore,
                'position': self.position,
                'cumulative_pnl': pnl
            })
            
        return pd.DataFrame(results)

# Generate correlated assets with lead-lag
asset_a, asset_b = generate_correlated_assets(n_ticks=20000, correlation=0.95, lead_lag=5)

# Run pairs trading strategy
pairs_trader = HFTPairsTrader(
    lookback=100,
    entry_zscore=2.0,
    exit_zscore=0.5,
    position_size=100
)

pairs_results = pairs_trader.backtest(asset_a, asset_b)

print("HFT Pairs Trading Results")
print("=" * 60)
print(f"Total Trades: {len([t for t in pairs_trader.trades if t['action'] == 'enter'])}")
print(f"Final PnL: ${pairs_results['cumulative_pnl'].iloc[-1]:,.2f}")
print(f"Max PnL: ${pairs_results['cumulative_pnl'].max():,.2f}")
print(f"Max Drawdown: ${pairs_results['cumulative_pnl'].min():,.2f}")

In [ ]:
# Visualize pairs trading
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Plot 1: Asset prices
ax1 = axes[0]
ax1.plot(pairs_results['tick'], pairs_results['price_a'], label='Asset A (Leader)', alpha=0.8)
ax1.plot(pairs_results['tick'], pairs_results['price_b'], label='Asset B (Lagger)', alpha=0.8)
ax1.set_xlabel('Tick')
ax1.set_ylabel('Price')
ax1.set_title('Correlated Assets with Lead-Lag Relationship')
ax1.legend()

# Plot 2: Spread and Z-score
ax2 = axes[1]
ax2_twin = ax2.twinx()
ax2.plot(pairs_results['tick'], pairs_results['spread'], color='purple', alpha=0.7, label='Spread')
ax2_twin.plot(pairs_results['tick'], pairs_results['zscore'], color='orange', alpha=0.7, label='Z-score')
ax2.axhline(0, color='black', linestyle='--', alpha=0.3)
ax2_twin.axhline(2, color='red', linestyle=':', alpha=0.5)
ax2_twin.axhline(-2, color='red', linestyle=':', alpha=0.5)
ax2_twin.axhline(0.5, color='green', linestyle=':', alpha=0.5)
ax2_twin.axhline(-0.5, color='green', linestyle=':', alpha=0.5)

ax2.set_xlabel('Tick')
ax2.set_ylabel('Spread', color='purple')
ax2_twin.set_ylabel('Z-score', color='orange')
ax2.set_title('Spread and Z-Score with Entry/Exit Thresholds')

# Mark trade entries and exits
entries = [t for t in pairs_trader.trades if t['action'] == 'enter']
exits = [t for t in pairs_trader.trades if t['action'] == 'exit']
for entry in entries[:20]:  # Show first 20
    ax2.axvline(entry['tick'], color='green', alpha=0.3, linestyle='-')
for exit in exits[:20]:
    ax2.axvline(exit['tick'], color='red', alpha=0.3, linestyle='-')

# Plot 3: Cumulative PnL
ax3 = axes[2]
ax3.plot(pairs_results['tick'], pairs_results['cumulative_pnl'], color='blue', linewidth=1.5)
ax3.fill_between(pairs_results['tick'], 0, pairs_results['cumulative_pnl'],
                  where=pairs_results['cumulative_pnl'] > 0, color='green', alpha=0.3)
ax3.fill_between(pairs_results['tick'], 0, pairs_results['cumulative_pnl'],
                  where=pairs_results['cumulative_pnl'] < 0, color='red', alpha=0.3)
ax3.axhline(0, color='black', linestyle='--', alpha=0.5)
ax3.set_xlabel('Tick')
ax3.set_ylabel('Cumulative PnL ($)')
ax3.set_title('Pairs Trading Strategy Performance')

plt.tight_layout()
plt.show()

## 7. Momentum Ignition Detection

### 7.1 Understanding Market Manipulation

Momentum ignition and spoofing are manipulative practices:
- **Momentum Ignition**: Placing aggressive orders to trigger price movement
- **Spoofing**: Placing orders with intent to cancel before execution
- **Layering**: Placing multiple orders at different price levels to create false impression

### 7.2 VPIN: Volume-Synchronized Probability of Informed Trading

VPIN measures order flow toxicity - the probability that counterparties are informed traders.

In [ ]:
class VPINCalculator:
    """
    Calculate Volume-Synchronized Probability of Informed Trading (VPIN).
    
    VPIN measures order flow toxicity based on the imbalance between
    buy and sell volumes in fixed-volume buckets.
    """
    
    def __init__(self, bucket_size: int = 1000, n_buckets: int = 50):
        """
        Args:
            bucket_size: Volume per bucket
            n_buckets: Number of buckets for VPIN calculation
        """
        self.bucket_size = bucket_size
        self.n_buckets = n_buckets
        
    def classify_trades(self, trade_data: pd.DataFrame) -> pd.DataFrame:
        """
        Classify trades as buy or sell initiated using tick rule.
        
        Tick rule:
        - If trade price > previous trade price: buy
        - If trade price < previous trade price: sell
        - If unchanged: use previous classification
        """
        df = trade_data.copy()
        df['price_change'] = df['trade_price'].diff()
        
        # Classify based on price change
        df['trade_sign'] = np.where(df['price_change'] > 0, 1,
                                    np.where(df['price_change'] < 0, -1, 0))
        
        # Forward fill for unchanged prices
        df['trade_sign'] = df['trade_sign'].replace(0, np.nan).ffill().fillna(1)
        
        return df
    
    def create_volume_buckets(self, trade_data: pd.DataFrame) -> List[Dict]:
        """Create fixed-volume buckets from trade data."""
        df = self.classify_trades(trade_data.dropna(subset=['trade_price']))
        
        buckets = []
        current_bucket = {'buy_volume': 0, 'sell_volume': 0, 'start_idx': 0}
        cumulative_volume = 0
        
        for idx, row in df.iterrows():
            volume = row['trade_size']
            
            if row['trade_sign'] > 0:
                current_bucket['buy_volume'] += volume
            else:
                current_bucket['sell_volume'] += volume
                
            cumulative_volume += volume
            
            if cumulative_volume >= self.bucket_size:
                current_bucket['end_idx'] = idx
                current_bucket['total_volume'] = cumulative_volume
                buckets.append(current_bucket)
                
                # Start new bucket
                current_bucket = {'buy_volume': 0, 'sell_volume': 0, 'start_idx': idx}
                cumulative_volume = 0
                
        return buckets
    
    def calculate_vpin(self, trade_data: pd.DataFrame) -> pd.DataFrame:
        """Calculate VPIN over time."""
        buckets = self.create_volume_buckets(trade_data)
        
        if len(buckets) < self.n_buckets:
            print(f"Warning: Only {len(buckets)} buckets created, need {self.n_buckets}")
            self.n_buckets = len(buckets)
        
        vpin_values = []
        
        for i in range(self.n_buckets, len(buckets)):
            # Get last n_buckets
            recent_buckets = buckets[i-self.n_buckets:i]
            
            # Calculate absolute order imbalance
            total_imbalance = sum(abs(b['buy_volume'] - b['sell_volume']) 
                                  for b in recent_buckets)
            total_volume = sum(b['buy_volume'] + b['sell_volume'] 
                              for b in recent_buckets)
            
            vpin = total_imbalance / total_volume if total_volume > 0 else 0
            
            vpin_values.append({
                'bucket_idx': i,
                'vpin': vpin,
                'end_idx': buckets[i]['end_idx']
            })
            
        return pd.DataFrame(vpin_values)


class SpoofingDetector:
    """Detect potential spoofing and layering patterns."""
    
    def __init__(self, 
                 cancel_ratio_threshold: float = 0.9,
                 order_lifetime_threshold: float = 0.1,  # seconds
                 layer_count_threshold: int = 5):
        
        self.cancel_ratio_threshold = cancel_ratio_threshold
        self.order_lifetime_threshold = order_lifetime_threshold
        self.layer_count_threshold = layer_count_threshold
        
    def detect_high_cancel_ratio(self, order_events: pd.DataFrame) -> pd.DataFrame:
        """
        Detect accounts with unusually high cancel ratios.
        """
        # Group by account and calculate cancel ratio
        account_stats = order_events.groupby('account_id').agg({
            'event_type': lambda x: (x == 'cancel').sum() / len(x)
        }).rename(columns={'event_type': 'cancel_ratio'})
        
        account_stats['is_suspicious'] = account_stats['cancel_ratio'] > self.cancel_ratio_threshold
        return account_stats
    
    def detect_layering(self, order_book_snapshots: List[Dict]) -> List[int]:
        """
        Detect layering patterns in order book snapshots.
        
        Layering: Multiple orders at consecutive price levels from same source,
        often with similar sizes, placed and cancelled together.
        """
        suspicious_snapshots = []
        
        for i, snapshot in enumerate(order_book_snapshots):
            # Check bid side
            bid_sizes = [size for _, size in snapshot.get('bids', [])]
            ask_sizes = [size for _, size in snapshot.get('asks', [])]
            
            # Detect similar-sized orders at consecutive levels
            if len(bid_sizes) >= self.layer_count_threshold:
                bid_std = np.std(bid_sizes[:self.layer_count_threshold])
                bid_mean = np.mean(bid_sizes[:self.layer_count_threshold])
                if bid_mean > 0 and bid_std / bid_mean < 0.2:  # Low variation
                    suspicious_snapshots.append(i)
                    continue
                    
            if len(ask_sizes) >= self.layer_count_threshold:
                ask_std = np.std(ask_sizes[:self.layer_count_threshold])
                ask_mean = np.mean(ask_sizes[:self.layer_count_threshold])
                if ask_mean > 0 and ask_std / ask_mean < 0.2:
                    suspicious_snapshots.append(i)
                    
        return suspicious_snapshots


# Calculate VPIN on tick data
vpin_calc = VPINCalculator(bucket_size=500, n_buckets=30)
trades_only = tick_data[tick_data['trade_direction'] != 0].copy()
vpin_results = vpin_calc.calculate_vpin(trades_only)

print("VPIN Analysis")
print("=" * 60)
print(f"Average VPIN: {vpin_results['vpin'].mean():.4f}")
print(f"Max VPIN: {vpin_results['vpin'].max():.4f}")
print(f"Min VPIN: {vpin_results['vpin'].min():.4f}")
print(f"\n💡 VPIN Interpretation:")
print("   - VPIN > 0.3: Elevated toxicity (caution)")
print("   - VPIN > 0.5: High toxicity (significant adverse selection risk)")
print("   - VPIN > 0.7: Extreme toxicity (market stress)")

# Check current toxicity level
latest_vpin = vpin_results['vpin'].iloc[-1]
if latest_vpin > 0.5:
    print(f"\n⚠️  Current VPIN: {latest_vpin:.4f} - HIGH TOXICITY WARNING")
else:
    print(f"\n✅ Current VPIN: {latest_vpin:.4f} - Normal toxicity level")

In [ ]:
# Visualize VPIN and toxicity
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: VPIN over time
ax1 = axes[0]
ax1.plot(vpin_results['bucket_idx'], vpin_results['vpin'], color='purple', linewidth=1)
ax1.axhline(0.3, color='yellow', linestyle='--', label='Elevated (0.3)')
ax1.axhline(0.5, color='orange', linestyle='--', label='High (0.5)')
ax1.axhline(0.7, color='red', linestyle='--', label='Extreme (0.7)')
ax1.fill_between(vpin_results['bucket_idx'], 0, vpin_results['vpin'],
                  where=vpin_results['vpin'] > 0.5, color='red', alpha=0.3)
ax1.set_xlabel('Volume Bucket')
ax1.set_ylabel('VPIN')
ax1.set_title('Volume-Synchronized Probability of Informed Trading (VPIN)')
ax1.legend()
ax1.set_ylim(0, 1)

# Plot 2: VPIN distribution
ax2 = axes[1]
ax2.hist(vpin_results['vpin'], bins=50, color='purple', alpha=0.7, edgecolor='black')
ax2.axvline(vpin_results['vpin'].mean(), color='black', linestyle='--', 
            label=f"Mean: {vpin_results['vpin'].mean():.3f}")
ax2.axvline(0.3, color='yellow', linestyle=':', linewidth=2, label='Elevated')
ax2.axvline(0.5, color='orange', linestyle=':', linewidth=2, label='High')
ax2.set_xlabel('VPIN')
ax2.set_ylabel('Frequency')
ax2.set_title('VPIN Distribution')
ax2.legend()

plt.tight_layout()
plt.show()

## 8. Optimal Execution Algorithms (TWAP/VWAP)

### 8.1 The Execution Problem

When executing large orders, traders face a trade-off:
- **Market Impact**: Aggressive execution moves prices against you
- **Timing Risk**: Slow execution exposes you to price volatility

Optimal execution algorithms balance these factors.

### 8.2 TWAP (Time-Weighted Average Price)
Splits order evenly over time. Simple but doesn't adapt to volume patterns.

### 8.3 VWAP (Volume-Weighted Average Price)
Splits order proportionally to expected volume profile. Better for liquid stocks.